## Importing libraries
We start of by importing the libraries that we need for this project.

In [2]:
from sklearn import preprocessing
import PIL
from PIL import Image
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

2022-12-05 00:45:18.104241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Case 1: images anomaly detection
When we look at the pictures we see alot of dishes and interior/exterior photo's but there are also alot of unhelpfull pictures. These pictures include logo's, people and other random images. There isn't any benifit of having these pictures on a tripadvisor page. Therefor we call these anomalies, these pictures have no need to be on a tripadvisor page, as they bring no information to the customer. Because of this it would be nice to detect these images, this model could then be used to clear up the pages of tripadvisor. 

The detection of these images is called anomaly detection, there are 2 sorts of anomaly detection:
- **Outlier detection:** For this model we need a dataset with both standard pictures and anomaly pictures. 
- **Novelty detection:** For this nodel the trainingset consists only of the standard pictures. The trainingset must be labeled, so it the models are in a supervised fashion.

The dataset of tripadvisor is not labeled, therefor we could use outlier detection. Another choice can be to create a dataset of interiors, exteriors and dishes. This dataset would allow us to use **novelty detection**. For this case we will start of by looking at novelty detection.



sources: 1

### Construction the dataset

The dataset will look like a array of 3 d matrixes.

In [3]:
# todo adding labels
def imagesInFolderToDataset(path, width, height, channels):
    fileNameList = glob.glob(f"{path}*")
    images = []
    for fileName in tqdm(fileNameList, total=len(fileNameList)):
        try:

            img = Image.open(f"{fileName}")
            img_np = np.array(img.resize(( width, height ), channels ))
            if img_np.shape == (width,height,channels):
                images.append(img_np)

        except PIL.UnidentifiedImageError:
            pass
    return np.array(images)

In [4]:
SIZE = 128

restaurants     = imagesInFolderToDataset("Images/restaurant/", SIZE, SIZE, 3)
buffets         = imagesInFolderToDataset("Images/buffet/", SIZE, SIZE, 3)

# create test and train set
labeled_data            = np.concatenate((restaurants, buffets), axis=0)

x_train, x_validation   = train_test_split(labeled_data, random_state=0, train_size = 0.8)
x_test                  = imagesInFolderToDataset("tripadvisor_dataset/tripadvisor_images/",SIZE,SIZE,3)

# normalize the data
x_train         = x_train.astype(float) / 255.
x_validation    = x_validation.astype(float) / 255.
x_test          = x_test.astype(float) / 255.

100%|██████████| 15183/15183 [01:13<00:00, 205.56it/s]


Om te beginnen gaan we greyscale gebruiken, de reden hiervoor is dat we op deze manier de dimensionaliteit reduceren. Dit zorgt dat er minder kans is op overfitting. Fotos met rgb maken het wel mogelijk om een beter model te trainen maar hebben meer kans op overfitting, de oplossingen hiervoor zijn dim reduction of meer data.

In [5]:
x_train_greyscale       = np.array(list(map( lambda x: np.dot(x[...,:3], [0.2989, 0.587, 0.114]), x_train)))
x_validation_greyscale  = np.array(list(map( lambda x: np.dot(x[...,:3], [0.2989, 0.587, 0.114]), x_validation)))
x_test_greyscale        = np.array(list(map( lambda x: np.dot(x[...,:3], [0.2989, 0.587, 0.114]), x_test)))

In [6]:
## convalutional autoencoder
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(SIZE, SIZE, 1)),
      layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
      layers.MaxPooling2D((2, 2), padding='same'),
      layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
      layers.MaxPooling2D((2, 2), padding='same'),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
      layers.MaxPooling2D((2, 2), padding='same')])


    self.decoder = tf.keras.Sequential([
      layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
      layers.UpSampling2D((2, 2)),
      layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
      layers.UpSampling2D((2, 2)),
      layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
      layers.UpSampling2D((2, 2)),

      layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

2022-12-05 00:46:50.232849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## vragen voor de prof
- Gaan we beter voor 3 anomaly detectors die elk een checken of iets behoord tot een klasse of gaan we voor of kunnen we een dataset samenstellen met alles wat geen anomaly is?
- Als validation data moeten we data voorzien die geen anomaly is. Nemen we hier beast data uit de tripadvisor dataset die we zelf labelden?

In [7]:
history = autoencoder.fit(x_train_greyscale, x_train_greyscale,
        epochs=100,
        batch_size=64,
        validation_data=(x_validation_greyscale,x_validation_greyscale), 
        shuffle=True);

# TODO: voeg eenvoudige grafiek zoals bij rare man lijn 92

Epoch 1/20
5/5 [==============================] - 3s 285ms/step - loss: 0.0671
Epoch 2/20
5/5 [==============================] - 1s 276ms/step - loss: 0.0660
Epoch 3/20
5/5 [==============================] - 1s 264ms/step - loss: 0.0649
Epoch 4/20
5/5 [==============================] - 1s 293ms/step - loss: 0.0636
Epoch 5/20
5/5 [==============================] - 1s 281ms/step - loss: 0.0618
Epoch 6/20
5/5 [==============================] - 1s 283ms/step - loss: 0.0592
Epoch 7/20
5/5 [==============================] - 1s 229ms/step - loss: 0.0552
Epoch 8/20
5/5 [==============================] - 1s 227ms/step - loss: 0.0492
Epoch 9/20
5/5 [==============================] - 1s 232ms/step - loss: 0.0404
Epoch 10/20
5/5 [==============================] - 1s 221ms/step - loss: 0.0297
Epoch 11/20
5/5 [==============================] - 1s 221ms/step - loss: 0.0207
Epoch 12/20
5/5 [==============================] - 1s 224ms/step - loss: 0.0175
Epoch 13/20
5/5 [==============================] 

In de volgende stappen gaan we eens gaan kijken wat ons model gaat doen met onze trainig data. 
We kunnen hierbij een aantal foto's nemen en kijken wat er mee gebeurt, hoe ziet de foto voor het model eruit en erna

In [21]:
autoencoder(x_test_greyscale)

reconstructions = autoencoder.predict(x_test_greyscale)
test_loss = tf.keras.losses.mae(reconstructions.reshape((15169, 64, 64)), x_test_greyscale)

475/475 [==============================] - 11s 22ms/step


tf.Tensor(
[[0.25333957 0.16122881 0.13313665 ... 0.05664623 0.06242742 0.15147145]
 [0.08478696 0.06028627 0.0486029  ... 0.02293136 0.01819096 0.05586029]
 [0.14973615 0.10154403 0.10102472 ... 0.08609897 0.07386189 0.19971568]
 ...
 [0.31445347 0.14793462 0.12063675 ... 0.04115247 0.02700299 0.02348117]
 [0.15784918 0.10751213 0.07984237 ... 0.0391697  0.03365728 0.08139479]
 [0.16138367 0.02744524 0.0360656  ... 0.06118822 0.03753172 0.04562687]], shape=(15169, 64), dtype=float64)


Ze kunnen anomely detection opdelen in 2 verschillende delen:
- **Outlier detection:** Our input dataset contains examples of both standard events and anomaly events. These algorithms seek to fit regions of the training data where the standard events are most concentrated, disregarding, and therefore isolating, the anomaly events. Such algorithms are often trained in an unsupervised fashion (i.e., without labels). We sometimes use these methods to help clean and pre-process datasets before applying additional machine learning techniques.

- **Novelty detection:** Unlike outlier detection, which includes examples of both standard and anomaly events, novelty detection algorithms have only the standard event data points (i.e., no anomaly events) during training time. During training, we provide these algorithms with labeled examples of standard events (supervised learning). At testing/prediction time novelty detection algorithms must detect when an input data point is an outlier.

sources: 1



## Sources
1. https://pyimagesearch.com/2020/01/20/intro-to-anomaly-detection-with-opencv-computer-vision-and-scikit-learn/
2. https://www.tensorflow.org/tutorials/generative/autoencoder
3. https://github.com/bnsreenu/python_for_microscopists/blob/master/260_image_anomaly_detection_using_autoencoders/260_image_anomaly_detection_using_autoencoders.py#L130

https://www.guru99.com/autoencoder-deep-learning.html

